# Python with PostgreSQL & PostGIS

## Libraries and Settings

In [5]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/python_postgresql_postgis


## Create database connection

In [7]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Dispose the engine
engine.dispose()

('osm_switzerland',)


## List tables in database

In [3]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public';"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

geography_columns
geometry_columns
spatial_ref_sys
planet_osm_roads
planet_osm_point
planet_osm_line
planet_osm_polygon


## Show columns and data types of selected table

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Dispose the engine
engine.dispose()

# Print the DataFrame
df

## Query: Select buildings for which full address is available in defined zip code areas

In [17]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query 
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p."addr:street" IS NOT NULL
                AND p."addr:housenumber" IS NOT NULL
                AND p."addr:city" IS NOT NULL
                AND p."addr:postcode" IN ('8400', '8400')"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


,osm_id,addr:street,addr:housenumber,addr:city,addr:postcode,building,geom
0,73502139,Gatterstrasse,9,Winterthur,8400,yes,"POLYGON ((8.74926 47.47481, 8.7493 47.47472, 8..."
1,68254705,Bruderhausstrasse,1,Winterthur,8400,yes,"POLYGON ((8.72605 47.47717, 8.72606 47.4771, 8..."
2,68254702,Hintere Finsteristrasse,2,Winterthur,8400,yes,"POLYGON ((8.72546 47.47737, 8.72556 47.47736, ..."
3,68254709,Bruderhausstrasse,3,Winterthur,8400,yes,"POLYGON ((8.72607 47.47738, 8.72608 47.47727, ..."
4,68254713,Bruderhausstrasse,3a,Winterthur,8400,yes,"POLYGON ((8.72737 47.47766, 8.7274 47.47758, 8..."
...,...,...,...,...,...,...,...
1270,174978171,Schaffhauserstrasse,152,Winterthur,8400,retail,"POLYGON ((8.71934 47.51914, 8.71941 47.51901, ..."
1271,43011361,Buchackerstrasse,54,Winterthur,8400,school,"POLYGON ((8.71652 47.5168, 8.71657 47.51672, 8..."
1272,1055794222,Rütihofstrasse,66,Winterthur,8400,yes,"POLYGON ((8.71478 47.51517, 8.7148 47.51516, 8..."
1273,1055794223,Rütihofstrasse,68,Winterthur,8400,yes,"POLYGON ((8.71495 47.5153, 8.71504 47.51524, 8..."


## Show selected features on map

<span style="color: blue;">Note the popup field in the map, which has been added to provide additional information about buildings.</span>

<span style="color: blue;">Example of alternative background maps (maptiles) are:</span>
- <span style="color: blue;">EsriWorldImagery</span>
- <span style="color: blue;">EsriWorldTopoMap</span>
- <span style="color: blue;">EsriWorldGrayCanvas</span>
- <span style="color: blue;">CartoDBDarkMatter</span>
- <span style="color: blue;">CartoDBPositron</span>


In [18]:
# Extract the longitude and latitude coordinates to define the center of the map
lon = gdf.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15,
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf,
    name='geojson',
    weight=0.5,
    fill_color='greenyellow',
    fillOpacity=0.8,
    popup=folium.GeoJsonPopup(fields=['addr:street',
                                      'addr:housenumber',
                                      'addr:city',
                                      'addr:postcode',
                                      'building'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select coffee stores within the administrative boundaries of a defined municipality

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """WITH winterthur_boundary AS (
            SELECT way
            FROM planet_osm_polygon
            WHERE boundary = 'administrative'
            AND admin_level = '8'
            AND name = 'Winterthur'
        )
        SELECT
            h.osm_id,
            h.shop,
            h.name,
            ST_Transform(h.way, 4326) AS geom
        FROM planet_osm_point h
        WHERE h.shop = 'coffee'
        AND ST_Within(ST_Transform(h.way, 4326), (SELECT ST_Transform(way, 4326) FROM winterthur_boundary));"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


## Show selected features on map

In [ ]:
# Extract the longitude and latitude coordinates to define the center of the map
lon = gdf.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'shop'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all supermarkets in a distance of 1000m around the central station in the city of Winterthur.

<span style="color: blue;">For each supermarket, the distance to the central station in meters is calculated and stored as new column 'distance_meters'.</span>

<span style="color: blue;">In addition, a popup field was added to the map, allowing users to view detailed information about each selected feature when they click on it.</span>

<span style="color: blue;">Note that WGS84 (World Geodetic System 1984) coordinates in ST_MakePoint(LON, LAT) can be derived from:</span>

- <span style="color: blue;">https://tools.retorte.ch/map</span>


In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            p.osm_id,
            p.shop,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'supermarket'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography,
                1000
            )
        ORDER BY distance_meters;"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


## Show selected features on map

In [ ]:
# Extract the longitude and latitude coordinates to define the center of the map
lon = gdf.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'distance_meters'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Create a query of all 'primary' roads within the administrative boundary of the city of Bern.

In [ ]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# SQL Abfrage erstellen
sql = """WITH bern_boundary AS (
            SELECT way
            FROM planet_osm_polygon
            WHERE boundary = 'administrative'
            AND admin_level = '8'
            AND name = 'Bern'
        )
        SELECT
            p.osm_id,
            p.highway,
            ST_TRANSFORM(p.way::geometry, 4326) AS geom
        FROM planet_osm_line AS p
        WHERE
            p.highway IN ('primary')
            AND ST_Within(
                ST_Transform(p.way, 4326),
                (SELECT ST_Transform(way, 4326) FROM bern_boundary)
            );"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf

## Show selected features on map

In [ ]:
# Extract the longitude and latitude coordinates to define the center of the map
lon = gdf.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='CartoDB positron')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all roads classified as 'primary' in Switzerland and create buffers around these roads

In [15]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query (major roads)
sql_a = """--Select major roads
           SELECT
           ST_TRANSFORM(p.way, 4326) AS geom
           FROM public.planet_osm_roads AS p
           WHERE
           highway IN ('primary')"""

# Query the database and store the result in a GeoDataFrame
gdf_a = gpd.GeoDataFrame.from_postgis(sql_a, engine, geom_col='geom')

# Define SQL query (buffers around major roads)
sql_b = """--Create buffers around major roads and combine these buffers to one single buffer
        SELECT 
        1 as group_id,
        ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 200)), 4326) AS combined_buffer_geom
        FROM public.planet_osm_roads AS p
        WHERE
        highway IN ('primary')"""

# Query the database and store the result in a GeoDataFrame
gdf_b = gpd.GeoDataFrame.from_postgis(sql_b, engine, geom_col='combined_buffer_geom')

# Dispose the engine
engine.dispose()

## Show selected features on map

In [ ]:
# Extract the longitude and latitude coordinates to define the center of the map
# lon = gdf.geometry.apply(lambda line: line.centroid.x).mean()
# lat = gdf.geometry.apply(lambda line: line.centroid.y).mean()

# Use city of Zürich as the center of the map
lon = 8.54104
lat = 47.37445

# Initialize the map (use grayscale tiles for better contrast)
m = folium.Map(location=[lat, lon], 
               zoom_start=14, 
               tiles='CartoDB positron')

# Add buffer to map
buffer_group = folium.FeatureGroup(name='Buffer')
folium.Choropleth(
    geo_data=gdf_b,
    fill_color='greenyellow'
).add_to(buffer_group)
buffer_group.add_to(m)

# Add roads to map
roads_group = folium.FeatureGroup(name='Roads')
folium.GeoJson(
    gdf_a,
    style_function=lambda feature: {
        'color': 'red',
        'weight': 3
    }
).add_to(roads_group)
roads_group.add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select 'primary' roads, create buffers around these roads and select all buildings within buffers

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define the query
sql = """WITH buffer AS (
                -- Create buffers around major roads and combine these buffers to one single buffer
                SELECT 
                        1 as group_id,
                        ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 200)), 4326) AS combined_buffer_geom
                        FROM public.planet_osm_roads AS p
                WHERE
                        -- Filter for major roads
                        highway IN ('primary')
                        )
        -- Select all buildings within the buffer
        SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                ST_Transform(p.way, 4326) AS geom
                FROM
                public.planet_osm_polygon AS p, buffer
        WHERE
                p."addr:city" IN ('Zürich')
                -- Filter for buildings within the buffer
                AND ST_Contains(buffer.combined_buffer_geom, ST_Transform(p.way, 4326));
        """

# Query the database and store the result in a GeoDataFrame
gdf_c = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Dispose the engine
engine.dispose()

gdf_c.head()

## Show selected features on map

In [ ]:
# Extract the longitude and latitude coordinates to define the center of the map
# lon = gdf.geometry.apply(lambda line: line.centroid.x).mean()
# lat = gdf.geometry.apply(lambda line: line.centroid.y).mean()

# Use city of Zürich as the center of the map
lon = 8.54104
lat = 47.37445

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=14, 
               tiles='CartoDB positron')

# Create feature groups
buffer_group = folium.FeatureGroup(name='Buffer')
roads_group = folium.FeatureGroup(name='Roads')
bldgs_group = folium.FeatureGroup(name='Buildings')

# Add buffer to map
folium.Choropleth(
    geo_data=gdf_b,
    fill_color='greenyellow'
).add_to(buffer_group)

# Add roads to map
folium.GeoJson(
    gdf_a,
    style_function=lambda feature: {
        'color': 'red',
        'weight': 3
    }
).add_to(roads_group)

# Add buildigs in buffer to map
folium.Choropleth(
    geo_data=gdf_c,
    name='map',
    fill_color='gray'
).add_to(bldgs_group)

# Add feature groups to map
m.add_child(buffer_group)
m.add_child(roads_group)
m.add_child(bldgs_group)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Calculate areas of Swiss municipalities and select all municipalities with an area >= 100 km2.

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Query the database    
sql = """WITH area_calculation AS (
        SELECT 
                osm_id,
                name,
                ST_Area(ST_Transform(way, 32632)) / 1000000 AS area_km2,
                ST_Transform(way, 4326) AS geom
        FROM planet_osm_polygon
        WHERE 
                boundary = 'administrative' 
                AND admin_level = '8'
        )
        SELECT *
        FROM area_calculation
        WHERE area_km2 >= 100;"""

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Close the connection
engine.dispose()

# Show query results
gdf


## Show selected features on map

In [ ]:
# Extract the longitude and latitude coordinates to define the center of the map
lon = gdf.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=8, 
               tiles='CartoDB positron')

# Map settings
folium.GeoJson(
    gdf,
    name='geojson',
    weight=1,
    fill_color='greenyellow',
    fillOpacity=0.5,
    popup=folium.GeoJsonPopup(fields=['name', 'area_km2'])
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Plot map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')